In [23]:
import os
import csv
import io
from zipfile import ZipFile
import zipfile


import pandas as pd
from google.cloud import bigquery
import glob
from google.oauth2 import service_account
import pandas_gbq


## 1. Extracting Zip Files

In [ ]:

# Replace 'your_zip_file.zip' with the actual name of your zip file
#zip_file_path = 'your_zip_file.zip'
#extract_folder = 'your_extraction_folder'  # Replace with the desired extraction folder

#with ZipFile(zip_file_path, 'r') as zip_ref:
    #zip_ref.extractall(extract_folder)

#print(f"Contents extracted to {extract_folder}")


In [ ]:
zip_folder = "/Users/biancabostrom/Documents/ADA/Wedge Project/WedgeZipOfZips"
extract_destination = "/Users/biancabostrom/Documents/ADA/Wedge Project/ExtractedFiles"

zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]

for zip_file in zip_files:
    zip_file_path = os.path.join(zip_folder, zip_file)

    with ZipFile(zip_file_path, 'r') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file = zf.open(file_name, 'r')
            input_file = io.TextIOWrapper(input_file, encoding="utf-8")

            for idx, line in enumerate(input_file):
                print(line)
                if idx > 3:
                    break

            input_file.close()  # tidy up

In [ ]:
# Set the directory path where your ZIP files are located
directory_path = r'C:\path\to\your\folder'  # Use a raw string for the path
 
# Iterate over all the files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.zip'):
        # Construct the full file path
        file_path = os.path.join(directory_path, filename)
 
        # Print the file path for debugging
        print(f"Attempting to extract: {file_path}")
 
        try:
            # Open the ZIP file
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                # Extract all the contents into the directory
                zip_ref.extractall(directory_path)
                print(f"Extracted {filename}")
        except Exception as e:
            print(f"Error extracting {filename}: {e}")
 
print("All files extracted.")

## 2. Cleaning files: headers, delimeters, nulls and quotes 

In [ ]:
#loop through each file to check and add headers if different 

correct_headers = [
    "datetime", "register_no", "emp_no", "trans_no", "upc", "description", "trans_type", "trans_subtype",
    "trans_status", "department", "quantity", "Scale", "cost", "unitPrice", "total", "regPrice", "altPrice",
    "tax", "taxexempt", "foodstamp", "wicable", "discount", "memDiscount", "discountable", "discounttype",
    "voided", "percentDiscount", "ItemQtty", "volDiscType", "volume", "VolSpecial", "mixMatch", "matched",
    "memType", "staff", "numflag", "itemstatus", "tenderstatus", "charflag", "varflag", "batchHeaderID", 
    "local", "organic", "display", "receipt", "card_no", "store", "branch", "match_id", "trans_id"
]

for this_zf in zip_files:
    with ZipFile(os.path.join(zip_folder, this_zf), 'a') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file_path = os.path.join(zip_folder, this_zf, file_name)

            # Read the content of the file
            with zf.open(file_name, 'r') as input_file:
                content = input_file.read().decode("utf-8")

            # Check if headers are present
            has_headers = all(header in content for header in correct_headers)

            if not has_headers:
                # Headers are missing, add them
                content = ",".join(correct_headers) + "\n" + content

                # Write the modified content back to the file
                zf.writestr(file_name, content.encode("utf-8"))

                print(f"Headers added to file {file_name}.")

            print(f"File {file_name} is good.")

print("Done checking and adding headers.")

In [ ]:
#look for delimters

delimiters = dict() 

# Start by reading in all the files again.

for this_zf in zip_files :
    with ZipFile(os.path.join(zip_folder, this_zf), 'r') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["It looks like",
                           file_name,
                           "has delimiter",
                           dialect.delimiter,
                           "."]))

            input_file.close() # tidy up

In [ ]:
#loop through each file and replace the delimters to "," in the files that don't

# Clean the files
for this_zf in zip_files:
    with ZipFile(os.path.join(zip_folder, this_zf), 'a') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file_path = os.path.join(zip_folder, this_zf, file_name)
            
            # Read the content of the file
            with zf.open(file_name, 'r') as input_file:
                content = input_file.read().decode("utf-8")
            
            # Check if the delimiter is not a comma
            if delimiters[file_name] != ",":
                # Replace the delimiter with a comma
                content = content.replace(delimiters[file_name], ",")

                # Write the modified content back to the file
                zf.writestr(file_name, content.encode("utf-8"))

                print(f"File {file_name} has been cleaned.")

print("Done cleaning.")

In [ ]:
#replace empty values with "null"

for this_zf in zip_files:
    with ZipFile(os.path.join(zip_folder, this_zf), 'a') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file_path = os.path.join(zip_folder, this_zf, file_name)

            # Read the content of the file
            with zf.open(file_name, 'r') as input_file:
                content = input_file.read().decode("utf-8")

            # Identify and replace null values (assuming nulls are represented as an empty string "")
            content = content.replace('""', 'null')

            # Write the modified content back to the file
            zf.writestr(file_name, content.encode("utf-8"))

            print(f"Null values handled in file {file_name}.")

print("Done checking and handling null values.")

## 3. Upload to GBQ

In [ ]:
service_path = "/Users/biancabostrom/Documents/ADA/Wedge\ Project/wedge-404400-cb3a632effa5.json"
service_file = 'wedge-404400-cb3a632effa5.json' 
gbq_proj_id = "wedge-404400" 
gbq_dataset_id = "wedge_data"


private_key =service_path + service_file

In [ ]:
credentials = service_account.Credentials.from_service_account_file("/Users/biancabostrom/Documents/ADA/Wedge Project/wedge-404400-cb3a632effa5.json")

client = bigquery.Client(credentials=credentials, project=gbq_proj_id)

In [ ]:
#Use the Pandas function to_gbq to upload your data to GBQ.

#Iterate through each CSV file in the folder

for file_name in os.listdir(zip_folder):
    if file_name.endswith('.csv'):
        file_path = os.path.join(zip_folder, file_name)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Define the BigQuery table ID in the format 'project.dataset.table'
        table_id = f'your_project.your_dataset.{file_name.replace(".csv", "")}'

        # Upload the DataFrame to BigQuery using to_gbq
        df.to_gbq(destination_table=table_id, project_id='your_project', if_exists='replace')

print("Upload complete.")
